# 02wk-006: 타이타닉, Autogluon (Fsize)

최규빈  
2023-09-12

# 1. 강의영상

<https://youtu.be/playlist?list=PLQqh36zP38-zEjYbXMD4e-nS0_-nx9Zac&si=eA9ME6Pe4ecmtIgi>

# 2. Import

``` python
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
```

    /kaggle/input/titanic/train.csv
    /kaggle/input/titanic/test.csv
    /kaggle/input/titanic/gender_submission.csv

In [1]:
#pip install autogluon

In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor

# 3. 분석의 절차

## A. 데이터

`-` 비유: 문제를 받아오는 과정으로 비유할 수 있다.

In [3]:
tr = TabularDataset("/kaggle/input/titanic/train.csv")
tst = TabularDataset("/kaggle/input/titanic/test.csv")

`-` 피처엔지니어링

In [4]:
tr.eval('Fsize = SibSp + Parch')
tst.eval('Fsize = SibSp + Parch')

## B. Predictor 생성

`-` 비유: 문제를 풀 학생을 생성하는 과정으로 비유할 수 있다.

In [5]:
predictr = TabularPredictor("Survived")

No path specified. Models will be saved in: "AutogluonModels/ag-20230912_115814/"

## C. 적합(fit)

`-` 비유: 학생이 공부를 하는 과정으로 비유할 수 있다.

`-` 학습

In [6]:
predictr.fit(tr.eval('Fsize = SibSp + Parch'))

Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20230912_115814/"
AutoGluon Version:  0.8.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #26~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Jul 13 16:27:29 UTC 2
Disk Space Avail:   294.00 GB / 490.57 GB (59.9%)
Train Data Rows:    891
Train Data Columns: 12
Label Column: Survived
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
    2 unique label values:  [0, 1]
    If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
    Available Memory:                    125936.3 MB
    Train Data 

`-` 리더보드확인 (모의고사채점)

In [8]:
predictr.leaderboard()

                  model  score_val  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2   0.860335       0.016608  2.934600                0.000400           0.236214            2       True         14
1        NeuralNetTorch   0.837989       0.008671  1.382263                0.008671           1.382263            1       True         12
2         LightGBMLarge   0.832402       0.003077  0.377827                0.003077           0.377827            1       True         13
3       NeuralNetFastAI   0.832402       0.007537  1.316123                0.007537           1.316123            1       True         10
4              CatBoost   0.826816       0.003649  0.528946                0.003649           0.528946            1       True          7
5               XGBoost   0.826816       0.004545  0.150251                0.004545           0.150251            1       True         11
6              LightGBM   0.821229

## D. 예측 (predict)

`-` 비유: 학습이후에 문제를 푸는 과정으로 비유할 수 있다.

`-` training set 을 풀어봄 (predict) $\to$ 점수 확인

In [10]:
(tr.Survived == predictr.predict(tr.eval('Fsize = SibSp + Parch'))).mean()

`-` test set 을 풀어봄 (predict) $\to$ 점수 확인 하러 캐글에 결과제출

In [12]:
tst.assign(Survived = predictr.predict(tst.eval('Fsize = SibSp + Parch'))).loc[:,['PassengerId','Survived']]\
.to_csv("autogluon(Fsize)_submission.csv",index=False)